In [1]:
!pip install -q transformers accelerate peft bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.7 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


In [3]:
MODEL_NAME = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

base_model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [4]:
def chat(model, prompt, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [5]:
print(chat(base_model,"Who survived Avada Kedavra and why?"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Who survived Avada Kedavra and why?

We've got the answers to all your questions in our Harry Potter and the Deathly Hallows: Part 2 review.

Harry Potter and the Deathly Hallows: Part 2 is now in theaters. Watch the movie's first trailer.

The final installment in the Harry Potter franchise is in theaters. Watch the film's new trailer.

The Deathly Hallows: Part 2 is in theaters. Watch the film's new trailer.

Harry Potter and the Deathly Hallows: Part 2 is in theaters. Watch the film's new trailer.

Harry Potter and the Deathly Hallows: Part 2 is now in theaters. Watch the film's new trailer.

Harry Potter and the Deathly Hallows: Part 2 is now in theaters. Watch the film's new trailer.

H


In [6]:
print(chat(base_model,"Explain the Harry Potter story in short."))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Explain the Harry Potter story in short.

Harry Potter is a series of fantasy novels written by British author J.K. Rowling. The books chronicle the adventures of a young wizard, Harry Potter, and his friends Ronald Weasley and Hermione Granger, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry’s struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard government, subjugate non-magical people, and destroy all non-pure-blood wizards.

How many books are there in the Harry Potter series?

There are 7 books in the Harry Potter series.

What is the order of the Harry Potter books?

The books in the Harry Potter series are ordered as follows:

1. Harry Potter and the Philosopher


In [7]:
print(chat(base_model,"which speel is used to fly a object"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


which speel is used to fly a object

i want to fly a object from one location to another location and then fly it back to the first location

thank you

I suggest you take a look at the following topics:

- http://www.arduino.cc/forum/index.php?topic=47470
- http://www.arduino.cc/forum/index.php?topic=27322
- http://www.arduino.cc/forum/index.php?topic=19592

A bit more searching on the forum would probably reveal some other threads that could be helpful too.

I suggest you take a look at the following topics:

- http://www.arduino.cc/forum/index.php?topic=47470
- http://www.arduino.cc/forum/index.php?


In [8]:
!unzip lora_output.zip -d lora_output

Archive:  lora_output.zip
   creating: lora_output/lora_output/
  inflating: lora_output/lora_output/tokenizer_config.json  
  inflating: lora_output/lora_output/README.md  
  inflating: lora_output/lora_output/tokenizer.json  
  inflating: lora_output/lora_output/adapter_model.safetensors  
   creating: lora_output/lora_output/checkpoint-235/
  inflating: lora_output/lora_output/checkpoint-235/optimizer.pt  
  inflating: lora_output/lora_output/checkpoint-235/tokenizer_config.json  
  inflating: lora_output/lora_output/checkpoint-235/README.md  
  inflating: lora_output/lora_output/checkpoint-235/tokenizer.json  
  inflating: lora_output/lora_output/checkpoint-235/trainer_state.json  
  inflating: lora_output/lora_output/checkpoint-235/scheduler.pt  
  inflating: lora_output/lora_output/checkpoint-235/adapter_model.safetensors  
  inflating: lora_output/lora_output/checkpoint-235/adapter_config.json  
  inflating: lora_output/lora_output/checkpoint-235/scaler.pt  
  inflating: lora_ou

In [9]:
ft_model = PeftModel.from_pretrained(
    base_model,
    "lora_output/lora_output",
)

ft_model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro

In [10]:
print(chat(ft_model,"Who survived Avada Kedavra and why?"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Who survived Avada Kedavra and why?
— HARRY POTTER
“Why?” repeated the snake. “Because I am Voldemort!”
Harry could not have been more astonished if the snake had told him
that it was an old family retainer who had come to return his wand. He
opened his mouth to ask, “You — you’re — ?” but the snake cut him off.
“I am he, I am he who has survived! I am the Dark Lord, I am the heir of
Slytherin, I am Lord Voldemort!”
“You — but —” said Harry. “But . . . I thought . . . you died . . . I
thought . . .”

“I did not die, Potter, I have been eighteen years in my tomb, I have been
reborn!” said the snake. “I am stronger than ever I was! I have powers



In [11]:
print(chat(ft_model,"Explain the Harry Potter story in short."))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Explain the Harry Potter story in short.
Harry Potter is a wizard who lives with his aunt, uncle, and cousin in a
town called Little Whinging. He is unaware that he is a wizard because
wizards are forbidden to use magic outside of school. But the Dursleys
don’t know Harry is a wizard.
Harry’s life changes forever when a giant called Hagrid comes to
collect him for Hogwarts School of Witchcraft and Wizardry. Harry has a
great time at Hogwarts, but he finds out that there is a dark side to
wizardry. A dark wizard called Lord Voldemort killed Harry’s parents when
Harry was a baby, and he tried to kill Harry too. But Voldemort’s power
was destroyed when he tried to kill Harry, and now he is weak.

Explain the story in more detail.



In [12]:
print(chat(ft_model,"which speel is used to fly a object"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


which speel is used to fly a object to a
certain place?




Sirius:
The right spell is the one that makes you want to go to a certain place.




Ron:
How do you get a thing to float?




Sirius:
It’s easier to get a thing to float than to fly. A thing that floats can’t be
weighed down. It’s not quite the same as a broom.
A broom can fly. It’s made of special wood and has a special
construction. But a thing that floats is just . . . light. It’s a thing that’s
very light.




Harry:
What do you have to do to make something that floats?




Sirius:
You have to make sure that it’s not weighed down by anything.




Ron


In [13]:
print(chat(base_model,"use of Wingardium Leviosa"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


use of Wingardium Leviosa.
“Crucio!”
“Riddikulus!”
“Imperio!”
“Muffliato!”
“Reducto!”
“Revelio!”
“Stupefy!”
“Accio!”
“Alohomora!”
“Diffindo!”
“Avada Kedavra!”
“Avada Kedavra!”

“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora!”
“Alohomora


In [14]:
print(chat(ft_model,"use of Wingardium Leviosa"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


use of Wingardium Leviosa, Harry
arrived at the foot of the stairs.
“The staircase to the Headmaster’s office,” said Fang, wagging his tail.
“You’re not supposed to be in here,” said Harry, who had never before seen
the place.
“I’m not,” said Fang. “I’m not allowed to go upstairs without the Head

Master. I’m supposed to stay here, but the door was open and I was bored,
so I thought I’d come and have a look around. I’ve never been in here before.
Look, there’s an old-fashioned fireplace with a real fire, and —”
But Fang broke off, his nose in the air, his ears flat on his head.
“What’s that smell?” he said.
“I don’t know,” said Harry.
But he knew. He knew it was


In [15]:
print(chat(ft_model,"it's Wingardium Leviosa"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


it's Wingardium Leviosa. . . .”
“Wingardium Leviosa,” repeated Hermione. “It’s a spell that means
‘I’m not touching it, I’m not touching it, I’m not touching it.’”
“Yeah, well, that’s a bit late now,” said Ron.
“I’m not touching it,” said Hermione, and Harry saw her hands
shaking. “It’s not my fault if it’s not working.”
“Yeah, well, it is your fault, because you were the one who tried to

make it move,” said Ron. “We’re not going to be able to do anything
about it, are we?”
“No,” said Hermione, still looking terrified.
“Well, it’s not that bad,” said Ron. “It’s not like it’s going to explode or
something. . . .”
Herm


In [16]:
print(chat(base_model,"it's Wingardium Leviosa"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


it's Wingardium Leviosa,” said Hermione,
thrusting the wand back into Harry’s hand.
“It’s Wingardium Leviosa, isn’t it?” said Harry, turning the wand
over in his hands, looking at it as though it were a very strange fish. “So
it’s not a wand at all, it’s a — a . . . a wandlike thing.”
“It’s a wand, all right,” said Ron. “And it’s a good one, look at it.”
“It’s a wand,” said Harry, still looking at it, as though he had never
seen it before. “It’s a wand. . . .”
“You’d better try it,” said Hermione. “I can’t see you getting into
Hogwarts without a wand.”
“Right,” said Harry, taking the wand and holding it in his right hand.
He
